In [11]:
from psycopg2.extras import execute_values

In [2]:
import wikidata as wd

In [27]:
import importlib
importlib.reload(wd)

<module 'wikidata' from '/Users/jaety/projects/jaesite/histowiki/etl/wikidata.py'>

In [28]:
ids = [1,2,3,4]

In [64]:
def update_table(table_create_template, table_insert_template, table_values):
    with wd.create_cursor() as cursor:
        cursor.execute(table_create_template)
        execute_values(cursor, table_insert_template, table_values)

In [65]:
def insert_qids(qids):
    sql_create = """
        CREATE TABLE IF NOT EXISTS person_qids (
            qid integer PRIMARY KEY
        )
    """
    sql_insert = """
        INSERT INTO person_qids (qid) VALUES %s
        ON CONFLICT (qid) DO NOTHING
    """
    update_table(sql_create, sql_insert, [(x,) for x in qids])

In [57]:
sql_create = """
    CREATE TABLE IF NOT EXISTS test_ids (
        id integer PRIMARY KEY,
        id2 integer
    )
"""
sql = """
    insert into test_ids (id,id2) VALUES %s
    ON CONFLICT (id) DO UPDATE
    SET id = excluded.id,
        id2= excluded.id2;
"""

In [62]:
insert_values = [(x,x+1) for x in ids]
with wd.create_cursor() as cursor:
    cursor.execute(sql_create)
    execute_values(cursor, sql,insert_values)

In [63]:
with wd.create_cursor() as cursor:
    cursor.execute("select * from test_ids")
    data = cursor.fetchall()
data

[(1, 2), (2, 3), (3, 4), (4, 5)]